In [1]:
import jieba
import re
import pymongo
from gensim import corpora, models, similarities
jieba.set_dictionary("dict.txt.big.txt")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [2]:
ettoday_list = []
for i in mycol.find({},{"title":1,"_id":0}):
    ettoday_list.append(i["title"])
print(ettoday_list[:10])

['快訊／高嘉瑜晚間好奇找阿北求證\u3000柯P終於「一句話」解答了', '美眉的夏天／最HOT名單出爐\u3000她激戰搶下冠軍抱千元大獎', '搭西寧軍艦南偵\u300022名教師登太平島深化主權意識', '桃園區慶讚中元水燈排車繞境\u3000傳承百年特色慶典', '鳳飛飛故事展、電影展…等紀念活動\u3000緬懷永遠的國民天后', '韓國瑜選總統8月3日從桃園出發\u3000要讓全國看到藍軍團結力量', '規劃「個人服裝包」\u3000國軍新兵服裝用APP快速搞定', '嘉義「火燈夜巡、大士爺祭」深獲在地認同\u3000下一步推向國際', '台民黨「不分區名單」她第一個被點名！\u3000柯文哲傾向不選才組黨', '影／她吃藥睡死…起床驚見「手被蟑螂咬」模樣超慘\u3000網加碼曝：而且非常痛']


In [3]:
def clean_data(title):
    sub_t = "\W+"
    t_sub = re.sub(sub_t,"",title)
    t_sub = re.sub("快訊","",t_sub)
    return list(jieba.cut(t_sub))

ettoday_list = [clean_data(t) for t in ettoday_list]

Building prefix dict from D:\program\semester_final\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u202fd84447beda33a25aefd770cfa382.cache
Loading model cost 1.267 seconds.
Prefix dict has been built succesfully.


In [4]:
corpora_dict_path = './model/corpora_dict.dict'
tfidf_model_path = "./model/tfidf.model"
index_path = "./model/simIndex.index"
dictionary = corpora.Dictionary(ettoday_list)
dictionary.save(corpora_dict_path)

corpus = [dictionary.doc2bow(text) for text in ettoday_list]
# corpora.MmCorpus.serialize('./corpus.mm', corpus) 

tfidf = models.TfidfModel(corpus)
tfidf.save(tfidf_model_path)

index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.token2id))
index.save(index_path)

****

In [1]:
import jieba
import re
import pymongo
from gensim import corpora, models, similarities
jieba.set_dictionary("dict.txt.big.txt")

In [2]:
corpora_dict_path = './model/corpora_dict.dict'
tfidf_model_path = "./model/tfidf.model"
index_path = "./model/simIndex.index"

tfidf = models.TfidfModel.load(tfidf_model_path)
dictionary = corpora.Dictionary.load(corpora_dict_path)
index = similarities.SparseMatrixSimilarity.load(index_path)

In [5]:
def getSims(sentence):
    texts = jieba.cut(sentence)
    new_vec = dictionary.doc2bow(texts)
    new_vec_tfidf = tfidf[new_vec]
    return index[new_vec_tfidf]

In [6]:
sims = getSims("震驚柯P宣布組黨台灣民眾黨成立")
print(sims[:10])

[0.18088523 0.         0.         0.         0.         0.
 0.         0.         0.11147286 0.        ]
